In [10]:
from phi.flow import *

In [11]:
def simulateFlow(den, vel, GRAVITY, DT):
    den = advect.mac_cormack(den, vel, dt=DT)
    gravity_force = den * GRAVITY * DT @ vel
    vel = advect.semi_lagrangian(vel, vel, dt=DT) + gravity_force
    vel, _ = fluid.make_incompressible(vel)
    return den, vel

## Simulator Parameters

In [12]:
size = 128
batchSize = 16
timeSteps = 20

GRAVITY = math.tensor([0, -9.81])
DT = 1
xMax = size
yMax = size
boundsBox=Box[0:xMax, 0:yMax]
activeBoundsBox=Box[1:xMax-1, 1:yMax-1]

## Run Simulation and Write

In [13]:
for i in range(batchSize):
    scene = Scene.create("./data/size" + str(size))
    
    den = CenteredGrid(Noise(), extrapolation.ZERO, x=xMax-2, y=yMax-2, bounds=activeBoundsBox)
    den = den.with_values(math.maximum(den.values, 0))

    vel = StaggeredGrid(0, extrapolation.ZERO, x=xMax-2, y=yMax-2, bounds=activeBoundsBox)
    vel += den * GRAVITY * DT @ vel
    vel, _ = fluid.make_incompressible(vel)

    #we probably don't need to include this part unless boundaries are unique
    #bound = CenteredGrid([0] * batchSize, extrapolation.ONE, x=xMax-2, y=yMax-2, bounds=activeBoundsBox)
    #bound = CenteredGrid(bound, x=xMax, y=yMax, bounds=boundsBox) 

    for i in range(timeSteps):
        scene.write({'water' : den, 'velocity' : vel}, frame=i)
        den, vel = simulateFlow(den, vel, GRAVITY, DT)

/home/austinteng/anaconda3/lib/python3.9/site-packages/phi/field/_scene.py:180: UserWarning: Failed to copy calling script to scene during Scene.create(): [Errno 2] No such file or directory: '/tmp/ipykernel_22964/4149452862.py'
  warnings.warn(f"Failed to copy calling script to scene during Scene.create(): {err}")
